# OpenAI Assistants
> Functionality for making calls to OpenAI Assistants

The OpenAIAssistantManager provides functionality for managing Assistants, threads, messages, and runs using the OpenAI API. 

It enables:

- Creating new Assistants with custom configurations
- Retrieving existing Assistants by ID
- Starting conversation threads connected to Assistants
- Sending user messages in threads  
- Running Assistants on threads and waiting for completion
- Retrieving responses and conversation history

This provides a convenient wrapper for leveraging OpenAI's conversational AI capabilities. Some key use cases:

- Creating Assistants tailored for specific applications like classifying text
- Gathering Assistant responses for datasets by managing threads 
- Building conversational bots that can contextually respond to users
- Orchestrating Assistant training through instruction tuning

The class handles setting up the necessary Assistant, thread, and run objects and pipeline. It abstracts away low-level API calls so users can focus on functionality.

Key methods include:

- `create_assistant` - Initialize new Assistants
- `create_thread` - Start thread linked to an Assistant 
- `send_message` - Add user messages to thread
- `run_assistant` - Get Assistant's response for a thread
- `get_response` - Retrieve replies from a thread

So in summary, the OpenAIAssistantManager provides a simple way to leverage OpenAI's powerful conversational models in an application.

In [ ]:
#| default_exp assistant
import time

In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import os
import getpass
import time

In [ ]:
#| export
class OpenAIAssistantManager:
    def __init__(self, client):
        self.client = client
        self.current_assistant = None
        self.current_thread = None

    def add_file(self, file_path, purpose='assistants'):
        with open(file_path, "rb") as file_data:
            return self.client.files.create(file=file_data, purpose=purpose)

    def create_assistant(self, name="Custom Teacher Utterances Classifier",
                            description="A tool for classifying teacher utterances into the categories OTR (opportunity to respond), PRS (praise), REP (reprimand), or NEU (neutral).",
                            instructions="""You are the co-founder of an ed-tech startup training an automated teacher feedback tool to classify utterances made. I am going to provide several sentences. 
                                            Please classify each sentence as one of the following: OTR (opportunity to respond), PRS (praise), REP (reprimand), or NEU (neutral)
        
                                            user: Can someone give me an example of a pronoun?
                                            assistant: OTR
                                            user: That's right, 'he' is a pronoun because it can take the place of a noun.
                                            assistant: PRS
                                            user: You need to keep quiet while someone else is reading.
                                            assistant: REP
                                            user: A pronoun is a word that can take the place of a noun.
                                            assistant: NEU

                                            Only answer with the following labels: OTR, PRS, REP, NEU""",
                            model="gpt-4-1106-preview",
                            tools=[{"type": "code_interpreter"}], file_id=None):

        assistant_kwargs = {
                "name": name,
                "description": description,
                "instructions": instructions,
                "model": model,
                "tools": tools if tools else []
            }

        if file_id:
            assistant_kwargs["file_ids"] = [file_id] if isinstance(file_id, str) else file_id

        self.current_assistant = self.client.beta.assistants.create(**assistant_kwargs)
        print(self.current_assistant.id)


    def create_custom_assistant(self, name="Custom Teacher Utterances Classifier",
                                description="A custom tool for classifying teacher utterances using gpt-3.5-turbo.",
                                instructions="""You are the co-founder of an ed-tech startup training an automated teacher feedback tool to classify utterances made. I am going to provide several sentences. 
                                            Please classify each sentence as one of the following: OTR (opportunity to respond), PRS (praise), REP (reprimand), or NEU (neutral)
                                            Only answer with the following labels: OTR, PRS, REP, NEU""",
                                model="gpt-3.5-turbo",
                                tools=[],
                                file_id=None):
        assistant_kwargs = {
            "name": name,
            "description": description,
            "instructions": instructions,
            "model": model,
            "tools": tools
        }

        if file_id:
            assistant_kwargs["file_ids"] = [file_id] if isinstance(file_id, str) else file_id
    
        self.current_assistant = self.client.beta.assistants.create(**assistant_kwargs)
        print(self.current_assistant.id)
        print(self.current_assistant.id)

    def retrieve_assistant(self, assistant_id):
        """Retrieves an existing Assistant by ID"""
        self.current_assistant = self.client.beta.assistants.retrieve(assistant_id)
        print(self.current_assistant.id)
    
    def create_thread(self, user_message, file_id=None):
        message = {
            "role": "user",
            "content": user_message
        }
        if file_id:
            message["file_ids"] = [file_id]

        self.current_thread = self.client.beta.threads.create(messages=[message])
        return self.current_thread

    
    def delete_thread(self, user_message, file_id=None):
        self.client.beta.threads.delete(thread_id=self.current_thread.id)
        return self.current_thread

    
    def send_message(self, message_content):
        if self.current_thread is None:
            raise Exception("No active thread. Create a thread first.")
        return self.client.beta.threads.messages.create(
            thread_id=self.current_thread.id,
            role = "user",
            content = message_content
        )
    
    def list_messages(self):
        if self.current_thread is None:
            raise Exception("No active thread. Create a thread first.")
        thread_messages = self.client.beta.threads.messages.list(thread_id=self.current_thread.id)
        return thread_messages
    
    def submit_message(self, user_message):
        """Sends a user message to the active thread.
        Takes the message content string as input."""
        if self.current_thread is None or self.current_assistant is None:
            raise Exception("Assistant and Thread must be initialized before submitting a message.")
        
        # Submit the user message
        self.client.beta.threads.messages.create(
            thread_id=self.current_thread.id, 
            role="user", 
            content=user_message
        )
        
        # Create a run for the submitted message
        run = self.client.beta.threads.runs.create(
            thread_id=self.current_thread.id,
            assistant_id=self.current_assistant.id,
        )

        # Wait for the run to complete before returning
        completed_run = self.wait_on_run(run)
        return completed_run
    

    def create_thread_and_run(self, user_input):
        # Create a new thread for each input
        self.current_thread = self.client.beta.threads.create()

        # Submit the message and wait for the run to complete
        run = self.submit_message(user_input)
        completed_run = self.wait_on_run(run)

        return self.current_thread, completed_run
    
    def wait_on_run(self, run):
        """Waits for a run to complete before returning."""
        import time

        while run.status == "queued" or run.status == "in_progress":
            run = self.client.beta.threads.runs.retrieve(
                thread_id=self.current_thread.id,
                run_id=run.id,
            )
            time.sleep(0.5)
        return run
    
    def retrieve_message(self, message_content):
        """Retrieves a message by ID"""
        if self.current_thread is None:
            raise Exception("No active thread. Create a thread first.")
        return self.client.beta.threads.messages.retrieve('message_id', thread_id=self.current_thread.id)

    def get_response(self):
        """Retrieves the response from the last run."""
        if self.current_thread is None:
            raise Exception("No active thread. Create a thread first.")
        return self.client.beta.threads.messages.list(thread_id=self.current_thread.id, order="asc")


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()